<a href="https://colab.research.google.com/github/anushanthbalachandran/ML_Coursework_Telco_Customer_Churn/blob/main/2425477_20241130_Anushanth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


plt.style.use("default")
sns.set_theme()
